In [2]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv('car.txt', header = None)
data.head(3)

,0,1,2,3,4,5,6
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc


In [4]:
data[0].value_counts()

high     432
low      432
vhigh    432
med      432
Name: 0, dtype: int64

In [5]:
#1.使用标签编码进行数据预处理
import sklearn.preprocessing as sp
train_data = pd.DataFrame([])#创建一组新的dataFrame，防止改动原来的数据，而且如果在原来的数据上改的话，
                             #下一次执行就会发生在这次执行之后的结果的基础上改的
lbes = {}
for k, v in data.items():
    lbe = sp.LabelEncoder()
    train_data[k] = lbe.fit_transform(v)
    lbes[k]=lbe

In [6]:
#2. 整理输入和输出
import sklearn.model_selection as ms
x, y = train_data.loc[:,:5], train_data[6]

In [7]:
#3. 拆分训练集和测试集，跳过因为数据集不够，再分的话容易造成欠拟合
#4. 选择模型和创建模型
import sklearn.ensemble as se
model = se.RandomForestClassifier(max_depth = 8, n_estimators = 200, random_state = 7)

In [8]:
#5. 使用交叉验证去验证模型
import sklearn.model_selection as ms
list_array = ms.cross_val_score(model, x, y, cv = 5, scoring='f1_weighted')
list_array.mean()

0.7665122680173463

In [ ]:
#6.1 验证曲线
params = np.arange(50, 550, 50)
train_scores, test_scores = ms.validation_curve(model, x, y, 'n_estimators', params, cv = 5)
print(test_scores)

e:\dev_kits_python\python-3.8.2\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass param_name=n_estimators, param_range=[ 50 100 150 200 250 300 350 400 450 500] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [10]:
#7.1 训练模型
model.fit(x, y)
pred_y = model.predict(x)

In [11]:
#7.2 测试模型
data = [['high', 'med', '5more', '4', 'big', 'low', 'unacc'],
        ['high', 'high', '4', '4', 'med', 'med', 'acc'],
        ['low', 'low', '2', '4', 'small', 'high', 'good'],
        ['low', 'med', '3', '4', 'med', 'high', 'vgood']]
test_data = pd.DataFrame(data)
for k,v in test_data.items():
    lbe = lbes[k]# 注意这里是用上面的标签编码器
    test_data[k] = lbe.transform(v) #这里不能用fit_transform，因为上面已经fit完了
test_x, test_y = test_data.loc[:, :5], test_data[6]
pred_test_y = model.predict(test_x)
print('预测值：', lbes[6].inverse_transform(pred_test_y)) #反转编码，标签编码器自带的
print('实际值：', lbes[6].inverse_transform(test_y.values))

预测值： ['unacc' 'acc' 'good' 'vgood']
实际值： ['unacc' 'acc' 'good' 'vgood']


In [12]:
#8. 模型评估、混淆矩阵、分类报告
#8.1 混淆矩阵
import sklearn.metrics as sm
array_confusion = sm.confusion_matrix(y, pred_y)
array_confusion

array([[ 382,    0,    1,    1],
       [  37,   27,    0,    5],
       [   5,    0, 1205,    0],
       [   1,    0,    0,   64]], dtype=int64)

In [13]:
#8.2 分类报告
cr= sm.classification_report(y, pred_y)
print(cr)

              precision    recall  f1-score   support

           0       0.90      0.99      0.94       384
           1       1.00      0.39      0.56        69
           2       1.00      1.00      1.00      1210
           3       0.91      0.98      0.95        65

    accuracy                           0.97      1728
   macro avg       0.95      0.84      0.86      1728
weighted avg       0.97      0.97      0.97      1728

